# Interface Segregation Principle

> You don't want to stick too many elements into an interface.

Let's create a `Machine` base class that defines a bunch of methods that we can use for multiple devices, such as printers, faxes, scanners, etc.

In [1]:
class Machine:
    def print(self, document):
        raise NotImplementedError()

    def fax(self, document):
        raise NotImplementedError()

    def scan(self, document):
        raise NotImplementedError()

Let's now implement a `MultiFunctionalPrinter` class that implements all of these methods.

In [2]:
class MultiFunctionPrinter(Machine):
    """This subclass has no problems inheriting from Machine"""
    def print(self, document):
        pass

    def fax(self, document):
        pass

    def scan(self, document):
        pass

So far so good, but let's try implementing an `OldFashionedPrinter` class now.

A regular printer cannot fax nor scan, so what should we do with those two methods?

In [3]:
class OldFashionedPrinter(Machine):
    def print(self, document):
        # imagine some code goes here instead of pass
        pass

    def fax(self, document):
        """ Option 1: do nothing """
        pass  # do-nothing

    def scan(self, document):
        """ Option 2: raise an error """
        raise NotImplementedError('Printer cannot scan!')

Both of these options are bad, because these methods will be available in the class API even though they are useless, and why would you make these methods available to someone else anyway if you're only going to confuse them or worse, induce them to error?

So what can we do? We split our original interface into smaller ones!

In [4]:
from abc import ABC, abstractmethod

class Printer(ABC):
    """Class for machines that print"""
    @abstractmethod
    def print(self, document): pass

class Scanner(ABC):
    """Class for machines that scan"""
    @abstractmethod
    def scan(self, document): pass

# same for Fax, etc.

The classes above are defined as *Abstract Base Classes* (ABCs). An ABC is a "blueprint" for other classes; an ABC cannot be instantiated by itself and needs to be implemented in other classes.

The `@abstractmethod` decorator is used to create abstract methods, which are definitions without implementation that must be implemented by subclasses.

Now that we have our smaller interfaces, we can inherit from them as needed, even from multiple of them at the same time.

In [5]:
# Inherit from a single base class
class MyPrinter(Printer):
    def print(self, document):
        print(document)

# Inherit from multiple base classes
class Photocopier(Printer, Scanner):
    def print(self, document):
        print(document)

    def scan(self, document):
        # something meaningful code should go here
        pass

There are instances where you may need a bigger interface for many functionalities. In that case, you can create another base class that inherits from multiple base classes.

In [6]:
class MultiFunctionDevice(Printer, Scanner):  # , Fax, etc
    @abstractmethod
    def print(self, document):
        pass

    @abstractmethod
    def scan(self, document):
        pass

And here we implement our new compound base class:

In [7]:
class MultiFunctionMachine(MultiFunctionDevice):
    def __init__(self, printer, scanner):
        """This is a decorator pattern; we'll see it in other lessons"""
        self.printer = printer
        self.scanner = scanner

    def print(self, document):
        self.printer.print(document)

    def scan(self, document):
        self.scanner.scan(document)

Let's now see what would happen with our original interface if we try to run it:

In [8]:
printer = OldFashionedPrinter()
printer.fax(123)  # nothing happens
printer.scan(123)  # oops!

NotImplementedError: Printer cannot scan!